In [23]:
# abc try
import math
from abc import ABC, abstractmethod

class Rockmass(ABC):

    def __init__(self, ucs, fa):
        self.ucs=ucs
        self.fa=fa

    @property
    @abstractmethod
    def fa(self) -> float:
        """Force subclass to return its specific backing store."""
        pass

    @fa.setter
    @abstractmethod
    def fa(self, value):
        """Centralized Forensic Audit: All friction angles must be physically possible."""
        if not (0 <= value <= 90):
            raise ValueError("Physics Breach: Friction angle must be between 0 and 90.")
        #self._fa = value; not this because abc class is not store

    @abstractmethod
    def estimated_strength(self, **kwargs):

        confinement = kwargs.get('confinement')

        k = (1 + math.sin(math.radians(self.fa))) / (1 - math.sin(math.radians(self.fa)))
        return self.ucs + (k * confinement) #type: ignore
    # basically all rockmass must have these logic same


class Hrock(Rockmass):
    """himalayan rock"""

    def __init__(self, ucs, fa, fd):
        self._fa = None
        super().__init__(ucs, fa)
        self.fd = fd


    @property
    def fa(self) -> float:
        return self._fa #type:ignore

    @fa.setter
    def fa(self,value):

        Rockmass.fa.fset(self, value) #type: ignore
        self._fa = value

    def estimated_strength(self, **kwargs):

        base_str = super().estimated_strength(**kwargs)
        stress_dir = kwargs.get('stress_dir')

        if stress_dir is not None and abs(stress_dir-self.fd)<=10:
            updated_base_str = base_str*0.68
            return updated_base_str
        return base_str



#shale = Hrock(18, 18, 180)  #180 on fa raise ValueError good

In [24]:
# to have an object that is something of class1 class2 and not entirely a single class

class TunnelSupport:
    """Different tunnel supports for different rockclass requiring different financial costs"""

    def __init__(self, cost_per_m: float, **kwargs ):
        shotcrete_thickness = kwargs.get('shotcrete_thickness')
        self.shotcrete_thickness=shotcrete_thickness
        bolt_length = kwargs.get('bolt_length')
        self.bolt_length = bolt_length
        self.cost_per_m = cost_per_m


class TunnelSection():
    def __init__(self, section_id: str, geology: Hrock, support: TunnelSupport):
        self.section_id = section_id
        self.geology = geology # a hrock obj
        self.support =support #tunnelsupport obj


    def calculate_risk_factor(self):
        # delegation; asking geology obj for its data

        strength = self.geology.estimated_strength(confinement=0, stress_dir=90)
        if strength<50:
            return "Low Risk"
        return "High Risk"

    def total_cost(self, length):
        return self.support.cost_per_m * length


In [25]:
# Create the Parts
granite = Hrock(ucs=150, fa=45, fd=45)
steel_ribs = TunnelSupport(shotcrete_thickness=None, bolt_length=20, cost_per_m=5000)
schist = Hrock(ucs=200, fa = 60, fd = 67)
shotcrete = TunnelSupport(shotcrete_thickness=200, bolt_length=None, cost_per_m=1000)


# Assemble the System

section_1 = TunnelSection("Chainage 0+000", geology=granite, support=steel_ribs)
section_2 = TunnelSection("Chainage0+500", geology=schist, support = shotcrete)

tunnel_sections = [section_1, section_2]

for sections in tunnel_sections:
    print(sections.total_cost(20))

100000
20000


In [26]:
class Engine():
    def __init__(self, horsepower):
        self.horsepower = horsepower

class DrillBit():
    def __init__(self, diameter):
        self.diameter = diameter

class DrillRig():

    def __init__(self, engine: Engine, bit: DrillBit):
        self.engine = engine
        self.bit = bit

    def power_density(self):
        return self.engine.horsepower/self.bit.diameter
